# Configuration Notebook
Useful for debugging configurations and viewing project configuration details.

## Setup
Configure defaults and select a project.

In [ ]:
# Set defaults
#default_projects_directory = '/home/dinalt/ai_assets/projects/experiments'
default_projects_directory = '../examples/trainers'
default_project = "dynamic_models"
config_template = ""

from ipyfilechooser import FileChooser
import os
fc = FileChooser(
    os.path.join(default_projects_directory, default_project), show_only_dirs=True,
    title="Select a Project Directory", select_default=True)
display(fc)

## Project Info

This cell loads the import dependencies and the project meta-data from the selected project directory.

In [ ]:
import sys, os
modules_path = os.path.join('..', 'src')
if modules_path not in sys.path: sys.path.insert(0, modules_path)
from pprint import pformat, pp
from IPython import display as ds
from forgather import Latent
from forgather.config import ConfigEnvironment
from forgather.codegen import generate_code
from forgather.yaml_encoder import to_yaml
from aiws.config import preprocessor_globals, MetaConfig
import aiws.notebooks as nb

assert os.path.exists(fc.selected_path), "Project directory does not exist."
nb.show_project_readme(fc.selected_path)

# Get meta-config for project
meta = MetaConfig(fc.selected_path)

nb.display_meta(meta, "### Meta Config\n")
nb.list_templates(meta.find_templates(meta.config_prefix), "### Available Configurations\n")

# Get default config for project
default_config = meta.default_config()
print('-' * 60)
print(f"Default Configuration: {default_config}")

# Get the full name of the selected config template in the template name-space.
# If empty, meta.config_path() will return the default template path.
config_template_path = meta.config_path(config_template)
print(f"Selected Template Name: {config_template_path}")

## List Available Templates
This will list all templates within the searchpath.

In [ ]:
def list_templates(prefix):
    nb.list_templates(meta.find_templates(prefix), "### Templates\n")
list_templates('')

### Show Referenced Templates

In [ ]:
nb.display_referenced_templates_tree(environment, config_template_path, "### Included Templates\n")

## Init Config Envrionment

In [ ]:
# Create configuration envrionment
environment = ConfigEnvironment(
    searchpath=meta.searchpath,
    global_vars=preprocessor_globals(fc.selected_path),
)

## Preprocess Configiguration
Not required, but can be useful for diagnostics prior to YAML parsing.

In [ ]:
pp_config = environment.preprocess(config_template_path)
display(ds.Markdown(f"#### Preprocessed Config\n" f"```yaml\n{pp_config}\n```\n"))

## Load Configuration

This will both preprocess and parse (YAML) the template in a single step, returning both the node-graph and the pre-processed config.

In [ ]:
config, pp_config = environment.load(config_template_path).get()

### Show Referenced Source Files

Show referenced sub-modules within the same package.  
For accurate results, the configuration must be instantiated.

In [ ]:
nb.display_referenced_source_list(config, "### Included Sources\n")

### Render Configuration as YAML

Note: The configuration graph is language independent. This merely translates the graph to YAML.

In [ ]:
display(ds.Markdown(f"### Loaded Configuration\n```yaml\n{to_yaml(config)}\n```"))

### Render Configuration as Python

This will display the configuraiton graph as Python code. This even works correctly for recursively generated code.

While the render tries to faithfully generate code which is identical to what Latent.materialize(config) would do, it's an interpretation and may not always produce identical results.

One known issue is that LambdaNodes, which take arguments, are not rendered correctly. They work fine with 'materialize(),' but the lambdas in the generated code don't accept arguments from their caller. While fixable, doing so is fairly complicated, and the author lacks an infinite supply of time.


In [ ]:
generated_code = generate_code(config)
display(ds.Markdown(f"### Generated Source Code\n```python\n{generated_code}\n```"))

## Materialized Configuration

Instantiate the configuration from the definition.
This loads all of the referenced modules and instantiates the main output. Some configurations will run preprocessing when loaded, so this can take a moment.

And don't run this if you don't trust the source of the configuration!

In [ ]:
#from loguru import logger
#logger.enable("forgather.latent")

config, pp_config = environment.load(meta.config_path(config_template)).get()

# Note: We inject the pre-processed config as an argument, which can then be used to log this information.
main_output = Latent.materialize(config, pp_config=pp_config)['main']
pp(main_output)

## Execute Generate Code

This executes the generated code and calls 'construct()', the default factory function, to instantiate the configuration.

In theory, the output should be identitical to calling Latent.materialize(config), but there are know differences (see section on code generation for details). 

In [ ]:
exec(generated_code)
main_output = construct(pp_config=pp_config)['main']
pp(main_output)

### Run Configuration

Assuming that this the output object has a 'run' method (training scripts do), the following will run it.

For a more robust approach, see: [train.ipynb](train.ipynb)

In [ ]:
main_output.run()

### Cleanup
Note: These will show the target directory and ask for confirmation before proceeding.

#### Delete All

In [ ]:
nb.delete_dir(config.meta['models_dir'], "Delete all models in project")

#### Delete Configuration Output Directory
This will delete the model and logs for the current configuration.

In [ ]:
nb.delete_dir(config.meta['output_dir'], "Delete output directory")